# Start Program

In [ ]:
import os
import numpy as np
%matplotlib inline
import matplotlib.pylab as plt
import autokeras as ak

# Use the 0th GPU in the computer
os.environ["CUDA_VISIBLE_DEVICES"]="0"

#import tensorflow as tf
#tf.config.list_physical_devices('GPU')

In [ ]:
def norma2(input_data):
    """Normalize all data based on their own max values
    
    Parameters
    ==========
    
    input_data - (np.array)
        the full array of all 2D images to normalize based upon their own np.max value
        
        
    Returns
    =======
    The normalized data
    
    """
    
    maxies = np.max(input_data, axis=(1,2))
    if np.count_nonzero(maxies) != len(input_data):
        print('Error - Divide By Zero Found')
    output = np.divide(input_data, maxies[:,None, None])

    return output

In [ ]:
# Load in the already split Training and Test Sets 64x64 image dataset

xTrain = np.load('../training_data/noise_train_datatset_01.npy', allow_pickle=True)
yTrain = np.load('../training_data/noise_train_labels_01.npy', allow_pickle=True)

In [ ]:
# Obtain every nth data point for faster training
new_xTrain2 = []
new_yTrain2 = []

for kk, value in enumerate(xTrain):
    if kk%2 == 0:
        new_xTrain2.append(value)
        new_yTrain2.append(yTrain[kk])
        
new_xTrain2 = np.asarray(new_xTrain2)
new_yTrain2 = np.asarray(new_yTrain2)

xTrain = new_xTrain2
yTrain = new_yTrain2

new_xTrain2.shape, new_yTrain2.shape

((88128, 64, 64), (88128, 3))

In [ ]:
# Section out Perfect - Edge - Screw - Training Testing
perf_dat = []
edge_dat = []
screw_dat = []

perf_lab = []
edge_lab = []
screw_lab = []


for idx, labels in enumerate(yTrain):
    if labels[0] == 1:
        perf_lab.append(labels)
        perf_dat.append(xTrain[idx])
    
    elif labels[1] == 1:
        edge_lab.append(labels)
        edge_dat.append(xTrain[idx])
        
    elif labels[2] == 1:
        screw_lab.append(labels)
        screw_dat.append(xTrain[idx])

        
perf_dat = np.asarray(perf_dat)
edge_dat = np.asarray(edge_dat)
screw_dat = np.asarray(screw_dat)

perf_lab = np.asarray(perf_lab)
edge_lab = np.asarray(edge_lab)
screw_lab = np.asarray(screw_lab)


X_train = np.vstack((perf_dat, edge_dat, screw_dat))
y_train = np.vstack((perf_lab, edge_lab, screw_lab))

#print('ONLY HAS EDGE AND SCREW DATA!!!!')

from sklearn.model_selection import train_test_split
xTrain, xTest, yTrain, yTest = train_test_split(X_train, 
                                                y_train, 
                                                test_size = 0.15, 
                                                random_state = 0)

print(xTrain.shape, xTrain.dtype, xTest.shape, xTest.dtype)

(74908, 64, 64) float64 (13220, 64, 64) float64


In [ ]:
y_train = np.asarray(yTrain)
y_test = np.asarray(yTest)


X_train = norma2(xTrain)
X_test = norma2(xTest)


X_train = X_train.reshape((X_train.shape[0], 64, 64, 1))
X_test = X_test.reshape((X_test.shape[0], 64, 64, 1))

X_train = np.asarray(np.repeat(X_train, 3, 3))
X_test = np.asarray(np.repeat(X_test, 3, 3))


X_train.shape, y_train.shape, X_test.shape, y_test.shape, X_train.dtype, X_test.dtype

((74908, 64, 64, 3),
 (74908, 3),
 (13220, 64, 64, 3),
 (13220, 3),
 dtype('float64'),
 dtype('float64'))

In [ ]:
# Saves on RAM
del edge_dat
del screw_dat
del perf_dat

del new_xTrain2
del edge_lab
del screw_lab
del perf_lab
del new_yTrain2

# ImageDataGenerator



In [ ]:
# Set Values
rot_ran = 5
shear_range = 0.3
zoom_ran = [0.75, 1.25]
width_shift_ran = 0.10 #0.3
height_shift_ran = 0.10 #0.3
horz_flip = True
vert_flip = True
fill_mod = 'constant'
fill_val = 0.0


bright_ran = [0.9, 1.1]

from tensorflow.keras.preprocessing.image import ImageDataGenerator

def nan_check(img):
    """Used in post ImageDataGenerator processing to determine if there is a np.nan
    in the image
    
    Parameters
    ==========
    
    img - (np.array)
        2D image array
        
        
    Returns
    =======
    The input image, but will print out 'NAN IN ARRAY' for the users if found
    
    """
    if np.isnan(np.sum(xTest[0])):
        print('NAN IN ARRAY!!!')
        sys.stdout.flush()
    return img/255.0

def nan_check2(img):
    """Used in post ImageDataGenerator processing to determine if there is a np.nan
    in the image
    
    Parameters
    ==========
    
    img - (np.array)
        2D image array
        
        
    Returns
    =======
    The input image, but will print out 'NAN IN ARRAY' for the users if found
    
    """
    if np.isnan(np.sum(xTest[0])):
        print('NAN IN ARRAY!!!')
        sys.stdout.flush()
    return img

In [ ]:
train_datagen = ImageDataGenerator(

    rotation_range=rot_ran,
    shear_range=shear_range,
    zoom_range=zoom_ran,
    width_shift_range=width_shift_ran,
    height_shift_range=height_shift_ran,
    horizontal_flip=horz_flip,
    vertical_flip=vert_flip,
    
    brightness_range=bright_ran,

    fill_mode=fill_mod,
    cval=fill_val,

    
    preprocessing_function=nan_check,
    validation_split=0.1)


val_datagen = ImageDataGenerator(
    
    #rotation_range=rot_ran,
    #shear_range=shear_range,
    #zoom_range=zoom_ran,
    #width_shift_range=width_shift_ran,
    #height_shift_range=height_shift_ran,
    #horizontal_flip=horz_flip,
    #vertical_flip=vert_flip,
    
    fill_mode=fill_mod,
    cval=fill_val,
    
    preprocessing_function=nan_check2,
    validation_split=0.01)



In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape, X_train.dtype, X_test.dtype

((74908, 64, 64, 3),
 (74908, 3),
 (13220, 64, 64, 3),
 (13220, 3),
 dtype('float64'),
 dtype('float64'))

In [ ]:
import tensorflow as tf

In [ ]:
batch_size=32

train_gen = train_datagen.flow(X_train, y_train, 
                                 batch_size=batch_size, 
                                 shuffle=True,
                                 subset='training', seed=7)


v_gen = val_datagen.flow(X_test, y_test,
                                 batch_size=batch_size,
                                 subset='training', seed=7)



ds = tf.data.Dataset.from_generator(lambda: train_gen, 
    output_types=(tf.float32, tf.float32), 
    output_shapes=([None,64,64,3], [None,3])
)


v = tf.data.Dataset.from_generator(lambda: v_gen, 
    output_types=(tf.float32, tf.float32), 
    output_shapes=([None,64,64,3], [None,3])
)

# AutoKeras

In [ ]:
import tensorflow as tf
model = tf.keras.models.load_model('../untrained_model-Autokeras/')


def shuffle_weights(model, weights=None):
    """Randomly permute the weights in `model`, or the given `weights`.
    This is a fast approximation of re-initializing the weights of a model.
    Assumes weights are distributed independently of the dimensions of the weight tensors
      (i.e., the weights have the same distribution along each dimension).
    :param Model model: Modify the weights of the given model.
    :param list(ndarray) weights: The model's weights will be replaced by a random permutation of these weights.
      If `None`, permute the model's current weights.
    """
    if weights is None:
        weights = model.get_weights()
    weights = [np.random.permutation(w.flat).reshape(w.shape) for w in weights]
    # Faster, but less random: only permutes along the first dimension
    # weights = [np.random.permutation(w) for w in weights]
    model.set_weights(weights)

shuffle_weights(model)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Set some parameters
epochs = 100
batch_size = 32

save_dir = '100-200_train'
checkpoint_save_path = '/content/drive/MyDrive/Will_CDI_Project/paper/autok_training/'+save_dir+'/{epoch:02d}-{val_loss:.7f}.hdf5'
print(checkpoint_save_path)

checkpoints = ModelCheckpoint(filepath=checkpoint_save_path,
                              save_best_only=True,
                              save_weights_only=True,
                              save_freq='epoch')

history = model.fit_generator(ds,
      epochs=epochs,  #### set repeat in training dataset
      steps_per_epoch=len(train_gen),
      validation_data=v,
      validation_steps=len(v_gen), 
      callbacks=[checkpoints, ], 
      verbose=1)

val_loss = history.history['val_loss']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
acc = history.history['accuracy']

np.savez('/content/drive/MyDrive/Will_CDI_Project/paper/autok_training/100-200_hist/history.npz', val_loss=val_loss, val_acc=val_acc, loss=loss, acc=acc)

/content/drive/MyDrive/Will_CDI_Project/paper/autok_training/100-200_train/{epoch:02d}-{val_loss:.7f}.hdf5
Train for 2107 steps, validate for 409 steps
Epoch 1/100
2107/2107 [==============================] - 140s 66ms/step - loss: 0.0092 - accuracy: 0.9969 - val_loss: 0.0034 - val_accuracy: 0.9989
Epoch 2/100
2107/2107 [==============================] - 139s 66ms/step - loss: 0.0093 - accuracy: 0.9970 - val_loss: 0.0037 - val_accuracy: 0.9988
Epoch 3/100
2107/2107 [==============================] - 140s 66ms/step - loss: 0.0087 - accuracy: 0.9969 - val_loss: 0.0098 - val_accuracy: 0.9966
Epoch 4/100
2107/2107 [==============================] - 140s 66ms/step - loss: 0.0094 - accuracy: 0.9968 - val_loss: 0.0357 - val_accuracy: 0.9871
Epoch 5/100
2107/2107 [==============================] - 140s 66ms/step - loss: 0.0084 - accuracy: 0.9971 - val_loss: 0.0119 - val_accuracy: 0.9960
Epoch 6/100
2107/2107 [==============================] - 140s 66ms/step - loss: 0.0090 - accuracy: 0.9968 - 

# HandMade - OG


In [ ]:
# Basic TF Imports
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.models import Model


# MODELS AND OPTIMIZERS
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adagrad
from tensorflow.keras.optimizers import Adadelta
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.optimizers import Nadam

from tensorflow.keras.models import Sequential


Nc = 64
n_labels = 3

model = Sequential()
model.add(Conv2D(Nc, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(Nc,Nc, 3)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Dropout(0.25))
model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(n_labels, activation='softmax'))

  

opto = Adam() 
model.compile(optimizer=opto, loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Unnessessary for initial training
model.load_weights('../trained_model-Handmade/handmade_trained.hdf5')

# Handmade - Reviewers

In [ ]:
# Basic TF Imports
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.models import Model


# MODELS AND OPTIMIZERS
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adagrad
from tensorflow.keras.optimizers import Adadelta
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.optimizers import Nadam


from tensorflow.keras.models import Sequential


model = Sequential()

model.add(Conv2D(64, kernel_size=(3, 3),
                activation='relu',
                input_shape=(64, 64, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.5)) #0.5

for i in range(1):

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.5)) #0.5

model.add(Flatten())
model.add(Dense(128, activation='relu')) #128
#model.add(Dropout(0.25)) #0.25

model.add(Dense(3, activation='softmax'))

optimizer = 'adam'
learning_rate = 0.0005
if optimizer == "adam":
    optimizer = tf.optimizers.Adam(learning_rate=learning_rate)
elif optimizer == "sgd":
    optimizer = tf.optimizers.SGD(learning_rate=learning_rate)
elif optimizer=='rmsprop':
    optimizer = tf.optimizers.RMSprop(learning_rate=learning_rate)
else:
    raise ValueError("unexpected optimizer name: %r" % (optimizer_name,))
  
model.compile(optimizer=optimizer,
          loss='categorical_crossentropy',
          metrics=['accuracy'])

model.summary()

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Set some parameters
epochs = 100
batch_size = 32

save_dir = '100-200_train'
checkpoint_save_path = '/content/drive/MyDrive/Will_CDI_Project/paper/handmade/'+save_dir+'/{epoch:02d}-{val_loss:.7f}.hdf5'
print(checkpoint_save_path)

checkpoints = ModelCheckpoint(filepath=checkpoint_save_path,
                              save_best_only=True,
                              save_weights_only=True,
                              save_freq='epoch')

history = model.fit_generator(ds,
      epochs=epochs,  #### set repeat in training dataset
      steps_per_epoch=len(train_gen),
      validation_data=v,
      validation_steps=len(v_gen), 
      callbacks=[checkpoints, ], 
      verbose=1)

val_loss = history.history['val_loss']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
acc = history.history['accuracy']

np.savez('/content/drive/MyDrive/Will_CDI_Project/paper/handmade/100-200_hist/history.npz', val_loss=val_loss, val_acc=val_acc, loss=loss, acc=acc)

/content/drive/MyDrive/Will_CDI_Project/paper/handmade/100-200_train/{epoch:02d}-{val_loss:.7f}.hdf5
Epoch 1/100


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


2107/2107 [==============================] - 105s 47ms/step - loss: 0.2700 - accuracy: 0.8808 - val_loss: 0.1727 - val_accuracy: 0.9309
Epoch 2/100
2107/2107 [==============================] - 98s 47ms/step - loss: 0.2658 - accuracy: 0.8834 - val_loss: 0.1606 - val_accuracy: 0.9438
Epoch 3/100
2107/2107 [==============================] - 98s 47ms/step - loss: 0.2664 - accuracy: 0.8808 - val_loss: 0.1554 - val_accuracy: 0.9451
Epoch 4/100
2107/2107 [==============================] - 97s 46ms/step - loss: 0.2667 - accuracy: 0.8813 - val_loss: 0.1409 - val_accuracy: 0.9518
Epoch 5/100
2107/2107 [==============================] - 98s 46ms/step - loss: 0.2638 - accuracy: 0.8828 - val_loss: 0.1718 - val_accuracy: 0.9331
Epoch 6/100
2107/2107 [==============================] - 98s 46ms/step - loss: 0.2655 - accuracy: 0.8832 - val_loss: 0.1487 - val_accuracy: 0.9454
Epoch 7/100
2107/2107 [==============================] - 98s 47ms/step - loss: 0.2702 - accuracy: 0.8825 - val_loss: 0.1516 - val

# VGG16

In [ ]:
# Basic TF Imports
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model


# MODELS AND OPTIMIZERS
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adagrad
from tensorflow.keras.optimizers import Adadelta
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.optimizers import Nadam

In [ ]:
Nc = 64
n_labels = 3

base_model=VGG16(input_shape=(Nc,Nc,3),
                 include_top=False)

#base_model.layers.pop() #Pop out the current logistic layer
tmp = base_model.output

#Add a fully connected layer
tmp=Flatten()(tmp)

# and a logistic layer -- let's say we have 200 classes
predictions = Dense(n_labels, activation='softmax')(tmp)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

#Fix the first 17 layers
for layer in model.layers[:15]:
    layer.trainable = False 
    

opto = SGD()
    
model.compile(optimizer=opto, loss='categorical_crossentropy', metrics=['accuracy'])

58892288/58889256 [==============================] - 2s 0us/step


In [ ]:
# Unnessessary for initial training
model.load_weights('../trained_model-VGG16/VGG16_trained.hdf5')

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Set some parameters
epochs = 100
batch_size = 32

save_dir = '100-200_train'
checkpoint_save_path = '/content/drive/MyDrive/Will_CDI_Project/paper/vgg/'+save_dir+'/{epoch:02d}-{val_loss:.7f}.hdf5'
print(checkpoint_save_path)

checkpoints = ModelCheckpoint(filepath=checkpoint_save_path,
                              save_best_only=True,
                              save_weights_only=True,
                              save_freq='epoch')

history = model.fit_generator(ds,
      epochs=epochs,  #### set repeat in training dataset
      steps_per_epoch=len(train_gen),
      validation_data=v,
      validation_steps=len(v_gen), 
      callbacks=[checkpoints, ], 
      verbose=1)

val_loss = history.history['val_loss']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
acc = history.history['accuracy']

np.savez('/content/drive/MyDrive/Will_CDI_Project/paper/vgg/100-200_hist/history.npz', val_loss=val_loss, val_acc=val_acc, loss=loss, acc=acc)

/content/drive/MyDrive/Will_CDI_Project/paper/vgg/100-200_train/{epoch:02d}-{val_loss:.7f}.hdf5
Epoch 1/100
   3/2107 [..............................] - ETA: 1:53 - loss: 0.0128 - accuracy: 0.9896

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


2107/2107 [==============================] - 114s 54ms/step - loss: 0.0161 - accuracy: 0.9943 - val_loss: 0.0068 - val_accuracy: 0.9976
Epoch 2/100
2107/2107 [==============================] - 113s 53ms/step - loss: 0.0161 - accuracy: 0.9942 - val_loss: 0.0058 - val_accuracy: 0.9983
Epoch 3/100
2107/2107 [==============================] - 111s 53ms/step - loss: 0.0144 - accuracy: 0.9948 - val_loss: 0.0045 - val_accuracy: 0.9986
Epoch 4/100
2107/2107 [==============================] - 112s 53ms/step - loss: 0.0177 - accuracy: 0.9939 - val_loss: 0.0043 - val_accuracy: 0.9989
Epoch 5/100
2107/2107 [==============================] - 112s 53ms/step - loss: 0.0153 - accuracy: 0.9949 - val_loss: 0.0096 - val_accuracy: 0.9968
Epoch 6/100
2107/2107 [==============================] - 112s 53ms/step - loss: 0.0161 - accuracy: 0.9947 - val_loss: 0.0101 - val_accuracy: 0.9965
Epoch 7/100
2107/2107 [==============================] - 113s 54ms/step - loss: 0.0159 - accuracy: 0.9943 - val_loss: 0.0090

FileNotFoundError: ignored